In [ ]:
from PIL import Image, ImageDraw
import json

# Open the image
image = Image.open('/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/defect-detection-and-segment-deep-learning/class_data/Val/powder_uncover/image/powder_uncover_converted_ 0374.png')

# Load the bounding box data from the JSON file
with open('/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/defect-detection-and-segment-deep-learning/class_data/Val/powder_uncover/label/powder_uncover_converted_ 0374.json') as f:
    data = json.load(f)


if data['shapes'] != 1:
    for i in range(len(data['shapes'])):
        points = data['shapes'][i]['points']

        x1, y1 = points[0]
        x2, y2 = points[1]

        # Create an ImageDraw object
        draw = ImageDraw.Draw(image)
        
        # paint and write the text with the bounding box points
        # text
        draw.text((x1, y1), "x1, y1", fill='red')
        draw.text((x2, y2), "x2, y2", fill='red')
        
        # paint the bounding box
        draw.rectangle([x1, y1, x2, y2], outline='green')


        # Draw the bounding box on the image
        # draw.rectangle([x1, y1, x2, y2], outline='green')
image

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sys
import os
import json
import numpy as np
from PIL import Image

sys.path.append(os.path.realpath('..'))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_set_dir = '../defect-detection-and-segment-deep-learning/class_data/Train'
test_set_dir = '../defect-detection-and-segment-deep-learning/class_data/Val'

# Define the transformations to apply to the images
transform = torchvision.transforms.Compose([
    # resize the image to 224x224
    torchvision.transforms.Resize((224, 224)),
    
    # convert the rgb image to grayscale
    torchvision.transforms.Grayscale(num_output_channels=1),
    # convert the image to a tensor
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,), (0.5,))
])

# Define the custom dataset class
class DefectDetectionDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None, mask:bool=False):
        self.root_dir = root_dir
        self.transform = transform
        self.mask = mask
        self.class_names = ['powder_uncover', 'powder_uneven', 'scratch']
        self.types = ['image']
        self.image_filenames = []
        # bounding box
        self.labels = []
        self.mask_filenames = []
        for class_name in self.class_names:
            class_dir = os.path.join(root_dir, class_name)
            # concatenate the image, label and mask directories
            for type_name in self.types:
                type_dir = os.path.join(class_dir, type_name)
                for filename in os.listdir(type_dir):
                    if filename.endswith('.png'):
                        self.image_filenames.append(os.path.join(type_dir, filename))
                        # read the label file for bounding box position

                        self.mask_filenames.append(os.path.join(type_dir.replace('image', 'mask'), filename.replace('.png', '.png')))
                        self.labels.append(class_name)
    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image = Image.open(self.image_filenames[idx])
        mask = Image.open(self.mask_filenames[idx]).convert('L')
        
    
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        if self.mask == True:    
            return image, mask, label
        else:
            return image, label
    
# Load the training and test datasets
trainset = DefectDetectionDataset(root_dir=train_set_dir, transform=transform)
testset = DefectDetectionDataset(root_dir=test_set_dir, transform=transform)

batch_size = 32

# Define the data loaders
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1)

model = torchvision.models.resnet50()
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(trainset.class_names))

model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)




### project target
* You are asked to detect and segment the defects of manufacturing from given images. 
* You are given with  total 450  images and ground truth annotations(mask and bounding box positions).
* You can use the image processing skills as well as deep learning methods for this homework.

### Project steps
* Step 1: Data preprocessing
* Step 2: Data augmentation
* Step 3: Model training: dectection and segmentation models
* Step 4: Model evaluation with IoU and precision
* Step 5: Creat a GUI for defect detection and segmentation

In [ ]:
# Define the transformations to apply to the images
transform = torchvision.transforms.Compose([
    # resize the image to 224x224
    torchvision.transforms.Resize((224, 224)),
    
    # convert the rgb image to grayscale
    torchvision.transforms.Grayscale(num_output_channels=1),
    # convert the image to a tensor
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,), (0.5,))
])

# Define the custom dataset class
class DefectDetectionDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None, mask:bool=False):
        self.root_dir = root_dir
        self.transform = transform
        self.mask = mask
        self.class_names = ['powder_uncover', 'powder_uneven', 'scratch']
        self.types = ['image']
        self.image_filenames = []
        # bounding box
        self.labels = []
        self.mask_filenames = []
        for class_name in self.class_names:
            class_dir = os.path.join(root_dir, class_name)
            # concatenate the image, label and mask directories
            for type_name in self.types:
                type_dir = os.path.join(class_dir, type_name)
                for filename in os.listdir(type_dir):
                    if filename.endswith('.png'):
                        self.image_filenames.append(os.path.join(type_dir, filename))
                        # read the label file for bounding box position

                        self.mask_filenames.append(os.path.join(type_dir.replace('image', 'mask'), filename.replace('.png', '.png')))
                        self.labels.append(class_name)
    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image = Image.open(self.image_filenames[idx])
        mask = Image.open(self.mask_filenames[idx]).convert('L')
        
    
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        if self.mask == True:    
            return image, mask, label
        else:
            return image, label
    
# Load the training and test datasets
trainset = DefectDetectionDataset(root_dir=train_set_dir, transform=transform)
testset = DefectDetectionDataset(root_dir=test_set_dir, transform=transform)

batch_size = 32

# Define the data loaders
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1)

model = torchvision.models.resnet50()
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(trainset.class_names))

model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)



In [1]:
import json
from tqdm import tqdm
from PIL import Image
import os
import pandas as pd
import shutil
import pathlib


yolo_train_image_path = "../defect-detection-and-segment-deep-learning/yolov7/defect/images/train/"
yolo_val_image_path = "../defect-detection-and-segment-deep-learning/yolov7/defect/images/val/"
yolo_image_path = "../defect-detection-and-segment-deep-learning/yolov7/defect/"

pathlib.Path(yolo_train_image_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(yolo_val_image_path).mkdir(parents=True, exist_ok=True)

train_set_dir = '../defect-detection-and-segment-deep-learning/class_data/Train'
test_set_dir = '../defect-detection-and-segment-deep-learning/class_data/Val'

class_names = ['powder_uncover', 'powder_uneven', 'scratch']
types = ['image']
yolo_csv = pd.DataFrame(
    columns=["category", "x", "y", "w", "h", "image_name", "set_type", "image_path"])

rename = False
save = True
copy = True

# convert the label to yolo format


def yolo_format(convert_img_file: str, save_img_file_name: str, save: bool = False):

    label_path = convert_img_file.replace(
        "image", "label").replace(".png", ".json")
    image_name = convert_img_file.split("/")[-1]

    with open(label_path, "r") as file:
        json_file = json.load(file)

    width, height = Image.open(convert_img_file).size

    for annotation in json_file["shapes"]:
        if annotation["label"] == "powder_uncover":
            category_id = 0
        elif annotation["label"] == "powder_uneven":
            category_id = 1
        else:
            category_id = 2
        points = annotation["points"]

        # point[0] is left top of the bounding box
        # point[1] is right bottom of the bounding box

        x_min, y_min = points[0]
        x_max, y_max = points[1]

        # convert the bounding box to yolo format
        x = (x_min + (x_max-x_min)/2) * 1.0 / width
        y = (y_min + (y_max-y_min)/2) * 1.0 / height
        w = (x_max-x_min) * 1.0 / width

        # the height have nagative value, so we need to use abs() to get the positive value
        h = abs((y_max-y_min) * 1.0 / height)

        yolo_format_data = str(category_id) + " " + str(x) + \
            " " + str(y) + " " + str(w) + " " + str(h)

        yolo_csv.loc[len(yolo_csv)] = [category_id, x, y, w, h,
                                       image_name, save_img_file_name, convert_img_file]

        # save the bounding box and category_id to a text file
        # name is the same as the image name

        if save == True:

            with open(yolo_train_image_path.replace("images", "labels").replace("train", save_img_file_name) + image_name.replace(".png", ".txt"), "a") as file:
                file.write(yolo_format_data)
                file.write("\n")

    if save == True:
        # save the image path to a text file
        with open(yolo_image_path + f"{ save_img_file_name }.txt", "a") as file:
            file.write(f"./images/{ save_img_file_name }/" + image_name)
            file.write("\n")

# rename the image, label and mask files
def rename_files(dataset_dir: str):

    for dataset in [dataset_dir]:
        for class_name in class_names:
            class_dir = os.path.join(dataset, class_name)
            # concatenate the image, label and mask directories
            for type_name in types:
                type_dir = os.path.join(class_dir, type_name)
                for filename in tqdm(os.listdir(type_dir)):

                    # rename the image name with class + image name
                    # remove file name with space

                    if rename == True:
                        # rename the image name with class + image name
                        os.rename(
                            src=f"{type_dir}/{filename}", dst=f"{type_dir}/{class_name}_{filename.replace(' ', '').lower()}")

                        # rename the label name with class + label name
                        os.rename(src=f"{type_dir.replace('image', 'label')}/{filename.replace('.png', '.json')}",
                                  dst=f"{type_dir.replace('image', 'label')}/{class_name}_{filename.replace('.png', '.json').replace(' ', '').lower()}")

                        # rename the mask name with class + mask name
                        os.rename(src=f"{type_dir.replace('image', 'mask')}/{filename}",
                                  dst=f"{type_dir.replace('image', 'mask')}/{class_name}_{filename.replace(' ', '').lower()}")


if __name__ == "__main__":

    if rename == True:

        rename_files(dataset_dir=train_set_dir)
        rename_files(dataset_dir=test_set_dir)

    for dataset in [train_set_dir, test_set_dir]:
        for class_name in class_names:
            class_dir = os.path.join(dataset, class_name)
            # concatenate the image, label and mask directories
            for type_name in types:
                type_dir = os.path.join(class_dir, type_name)
                for filename in tqdm(os.listdir(type_dir)):

                    if dataset == train_set_dir:
                        yolo_format(
                            convert_img_file=f"{type_dir}/{filename}", save_img_file_name='train', save=save)
                    else:
                        yolo_format(
                            convert_img_file=f"{type_dir}/{filename}", save_img_file_name='val', save=save)

    if copy == True:
        for dataset in ["train", "val"]:
            for image_paths in yolo_csv.query(f"set_type == '{dataset}'")["image_path"].unique():
                # print(image_paths)
                shutil.copy(image_paths, yolo_image_path +
                            f"images/{dataset}/")
            print(f"copy {dataset} images done")


100%|██████████| 50/50 [00:01<00:00, 36.51it/s]


copy train images done
copy val images done


In [23]:
# using str.strip to remove space of with class name dictionary file name
import os, shutil
import pathlib

# recusive the following folder and save file name in list
Train_folder ='/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/DIP/final_project/defect-detection-and-segment-deep-learning/class_data/Val/poser_uneven/'

train_file_list = []

for root, dirs, files in os.walk(Train_folder):
    for file in files:
        train_file_list.append(file.strip().replace('.png', '').replace('.json', ''))

# 找出檔案出現次數小於 3 次的檔案
for i in train_file_list:
    if train_file_list.count(i) < 3:
        print(i)
        
# for i in train_file_list:
#     if train_file_list.count(i) > 3:
#         print(i)